In [27]:
# cell 1
import warnings
warnings.filterwarnings("ignore", message="CropBox missing from /Page")
import pdfplumber
import re
import pandas as pd
from pathlib import Path

# Path to your PDF
PDF_PATH = Path(r"C:\Users\DELL\Desktop\Karis\July 2024.pdf")
# Where to write your consolidated CSV
OUTPUT_CSV = Path("all_candidates.csv")


In [28]:
def parse_candidate_page(text: str) -> list[dict]:
    """
    Parse one page of text into a list of unit-records,
    each enriched with student info and registration number.
    """
    # 1) clean lines
    lines = [L.strip() for L in text.split("\n") if L.strip()]

    # 2) extract registration number & student name (e.g. "013001/EIN/... : Njue Daniel")
    m = re.match(r"^([0-9A-Z/]+)\s*:\s*(.+)$", lines[0])
    if m:
        reg_no       = m.group(1).strip()
        student_name = m.group(2).strip()
    else:
        reg_no       = "UNKNOWN"
        student_name = lines[0]

    # 3) level
    lvl_idx = next((i for i,L in enumerate(lines) if L.startswith("Level")), None)
    level = lines[lvl_idx].split()[1] if lvl_idx is not None else "UNKNOWN"

    # 4) course name
    course_line = next((L for L in lines[lvl_idx+1:] if ":" in L), "")
    course_name = course_line.split(":",1)[1].strip() if ":" in course_line else "UNKNOWN"

    # 5) collect unit lines until footer
    tbl_idx = next((i for i,L in enumerate(lines) if L.startswith("Unit Code")), None)
    if tbl_idx is None:
        return []
    raw = lines[tbl_idx+1:]
    unit_lines = []
    for L in raw:
        if re.fullmatch(r"\d+/\d+", L):  # page footer like "277/334"
            break
        unit_lines.append(L)

    # 6) parse units (handles multi-line names and split "Not Yet Competent")
    categories  = ["Basic Unit", "Core Unit", "Common Unit"]
    competences = ["Not Yet Competent", "Mastery", "Proficient", "Competent"]
    code_re     = re.compile(r"^[A-Z0-9]{2,}(?:/[A-Z0-9]+)+")
    cat_re      = re.compile(rf"\b({'|'.join(categories)})\b")
    one_word    = {"Mastery","Proficient","Competent"}

    units = []
    current = None
    pending_not_yet = False

    for L in unit_lines:
        if pending_not_yet:
            if L == "Competent":
                current["Competence"] = "Not Yet Competent"
                pending_not_yet = False
                continue
            pending_not_yet = False

        if current and L in one_word:
            current["Competence"] = L
            continue

        if current and L == "Not Yet":
            pending_not_yet = True
            continue

        if code_re.match(L):
            if current:
                units.append(current)
            code, rest = L.split(maxsplit=1)
            cm = cat_re.search(rest)
            category = cm.group(1) if cm else ""
            # strip out category
            if cm:
                rest = (rest[:cm.start()] + rest[cm.end():]).strip()
            comp = ""
            for cp in competences:
                if rest.endswith(cp):
                    comp = cp
                    rest = rest[:-len(cp)].strip()
                    break
            name = rest
            current = {
                "Registration Number": reg_no,
                "Student Name":        student_name,
                "Level":               level,
                "Course Name":         course_name,
                "Unit Code":           code,
                "Unit Name":           name,
                "Category":            category,
                "Competence":          comp
            }
        else:
            # continuation of unit name
            if current:
                current["Unit Name"] += " " + L

    if current:
        units.append(current)
    return units


In [29]:
all_records = []

table_settings = {
    "vertical_strategy":   "lines",
    "horizontal_strategy": "lines",
    "intersection_tolerance": 5,
}

with pdfplumber.open(PDF_PATH) as pdf:
    for page in pdf.pages:
        text = page.extract_text() or ""
        lines = [L.strip() for L in text.split("\n") if L.strip()]

        # --- Registration Number & Student Name ---
        # First non-empty line is like "013001/EIN/5/2024/004 : Njue Daniel Munene"
        reg_no = student = ""
        m0 = re.match(r"^([^:]+)\s*:\s*(.+)$", lines[0])
        if m0:
            reg_no  = m0.group(1).strip()
            student = m0.group(2).strip()

        # --- Level ---
        lvl_idx = next((i for i,L in enumerate(lines) if L.startswith("Level")), None)
        level = lines[lvl_idx].split(maxsplit=1)[1] if lvl_idx is not None else ""

        # --- Course Code & Name (between Level and Unit Code) ---
        tbl_idx = next((i for i,L in enumerate(lines) if L.startswith("Unit Code")), None)
        course_code = course_name = ""
        if lvl_idx is not None and tbl_idx is not None:
            for L in lines[lvl_idx+1:tbl_idx]:
                if re.match(r"^[0-9]{6}T.+?:", L):
                    parts = L.split(":", 1)
                    course_code = parts[0].strip()
                    course_name = parts[1].strip()
                    break

        # --- Extract the table of units ---
        table = page.extract_table(table_settings)
        if not table or len(table) < 2:
            continue

        for row in table[1:]:
            code, name, cat, comp = (cell or "" for cell in row)
            code = code.strip()
            name = " ".join(name.split())
            cat  = cat.strip()
            comp = " ".join(comp.split())

            if not code:
                continue

            all_records.append({
                "Registration Number": reg_no,
                "Student Name":        student,
                "Level":               level,
                "Course Code":         course_code,
                "Course Name":         course_name,
                "Unit Code":           code,
                "Unit Name":           name,
                "Category":            cat,
                "Competence":          comp
            })

df = pd.DataFrame(all_records)
print(f"Extracted {len(df)} rows from {len(pdf.pages)} pages")
df.head()

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Extracted 339 rows from 58 pages


,Registration Number,Student Name,Level,Course Code,Course Name,Unit Code,Unit Name,Category,Competence
0,013001/EIN/5/2024/004,Njue Daniel Munene,5,071305T4EIN,Electrical Installation,ENG/OS/EI/BC/03/5,Demonstrate Digital Literacy,Basic Unit,Competent
1,013001/EIN/5/2024/004,Njue Daniel Munene,5,071305T4EIN,Electrical Installation,ENG/OS/EI/CR/02/5,Perform Electrical Installation,Core Unit,Not Yet Competent
2,013001/EIN/5/2024/004,Njue Daniel Munene,5,071305T4EIN,Electrical Installation,ENG/OS/EI/CC/02/5,Perform Workshop Processes,Common Unit,Competent
3,013001/EIN/5/2024/004,Njue Daniel Munene,5,071305T4EIN,Electrical Installation,ENG/OS/EI/CC/04/5,Prepare and Interpret Technical Drawing,Common Unit,Not Yet Competent
4,013001/EIN/5/2024/004,Njue Daniel Munene,5,071305T4EIN,Electrical Installation,ENG/OS/EI/BC/01/5,Demonstrate Communication Skills,Basic Unit,Not Yet Competent


In [30]:
# cell 4
df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved to {OUTPUT_CSV.resolve()}")


Saved to C:\Users\DELL\Desktop\Karis\all_candidates.csv


In [31]:
# ─── Cell 4: Setup Numeric Scores & Flags ───────────────────────────────────────
# Map Competence → numeric score
score_map = {
    "Not Yet Competent": 1,
    "Competent":         2,
    "Proficient":        3,
    "Mastery":           4
}
df["Score"] = df["Competence"].map(score_map)

# Pass flag
df["Pass"] = df["Competence"].isin(["Competent","Proficient","Mastery"])

# Fail flag
df["Fail"] = ~df["Pass"]


In [32]:
# ─── Cell 5: I. Overall Exam Statistics & II. Competence Distribution ──────────
# I. Overall Examination Statistics
total_students   = df["Student Name"].nunique()
total_units      = len(df)
overall_pass_rate= df["Pass"].mean()
overall_fail_rate= df["Fail"].mean()

# II. Competence Level Distribution & Counts
comp_counts = (df["Competence"]
                .value_counts()
                .reindex(["Not Yet Competent","Competent","Proficient","Mastery"], fill_value=0)
                .rename_axis("Competence")
                .reset_index(name="Count"))

overall_pass_count = int(df["Pass"].sum())
overall_fail_count = int(df["Fail"].sum())

# Display
print("I. Overall Stats")
print(f"Total Students: {total_students}")
print(f"Total Units Assessed: {total_units}")
print(f"Overall Pass Rate: {overall_pass_rate:.1%}")
print(f"Overall Fail Rate: {overall_fail_rate:.1%}\n")

print("II. Competence Distribution")
print(comp_counts)
print(f"Overall Pass Count: {overall_pass_count}")
print(f"Overall Fail Count: {overall_fail_count}")


I. Overall Stats
Total Students: 58
Total Units Assessed: 339
Overall Pass Rate: 78.8%
Overall Fail Rate: 21.2%

II. Competence Distribution
          Competence  Count
0  Not Yet Competent     72
1          Competent    106
2         Proficient    130
3            Mastery     31
Overall Pass Count: 267
Overall Fail Count: 72


In [33]:
# ─── Cell 6: Compute Summaries & Save All Charts ───────────────────────────────

import matplotlib.pyplot as plt
from pathlib import Path

# Paths
BASE_DIR = Path(r"C:\Users\DELL\Desktop\Karis")
FIG_DIR  = BASE_DIR / "figs_all"
FIG_DIR.mkdir(exist_ok=True)

# I. Overall metrics (already in Cell 5)
# total_students, total_units, overall_pass_rate, overall_fail_rate

# assume overall_pass_count and overall_fail_count are integers
counts = [overall_pass_count, overall_fail_count]
labels = ["Pass", "Fail"]

plt.figure(figsize=(6,6))
plt.pie(counts, labels=labels, autopct="%1.1f%%", startangle=90)
plt.title("Overall Pass vs. Fail Distribution")
plt.tight_layout()
plt.savefig(FIG_DIR/"overall_pass_fail_pie.png")
plt.close()


# II. Competence distribution
comp_counts = (
    df["Competence"]
      .value_counts()
      .reindex(
        ["Not Yet Competent","Competent","Proficient","Mastery"],
        fill_value=0
      )
      .rename_axis("Competence")
      .reset_index(name="Count")
)

# III. Average Score by Category
avg_by_cat = (
    df.groupby("Category")["Score"]
      .mean()
      .round(2)
      .reset_index(name="Avg Score")
)

# IV. Pass rate by Course
course_pass = (
    df.groupby("Course Name")["Pass"]
      .mean()
      .mul(100)
      .round(1)
      .reset_index(name="Pass Rate (%)")
      .sort_values("Pass Rate (%)", ascending=False)
)

# V. Pass rate by Level
level_pass = (
    df.groupby("Level")["Pass"]
      .mean()
      .mul(100)
      .round(1)
      .reset_index(name="Pass Rate (%)")
      .sort_values("Level")
)

# VI. Pass rate by Category
category_pass = (
    df.groupby("Category")["Pass"]
      .mean()
      .mul(100)
      .round(1)
      .reset_index(name="Pass Rate (%)")
)

# VII. Course × Level breakdown
course_level = (
    df.groupby(["Course Name","Level"])
      .agg(
        Students = ("Student Name","nunique"),
        Pass_Rate =("Pass","mean")
      )
      .assign(Pass_Rate=lambda d: (d["Pass_Rate"]*100).round(1))
      .reset_index()
)

# VIII. Unit-specific pass rates
unit_pass = (
    df.groupby(["Unit Code","Unit Name"])["Pass"]
      .mean()
      .mul(100)
      .round(1)
      .reset_index(name="Pass Rate (%)")
      .sort_values("Pass Rate (%)")
)

# IX. Performance by Academic Course (counts + rates)
course_stats = (
    df.groupby("Course Name")
      .agg(
        Students   = ("Student Name", "nunique"),
        Pass_Count = ("Pass", "sum"),
        Fail_Count = ("Fail", "sum"),
        Pass_Rate  = ("Pass", "mean")
      )
      .assign(
        Pass_Rate=lambda d: (d["Pass_Rate"]*100).round(1),
        Fail_Rate=lambda d: (100-d["Pass_Rate"]).round(1)
      )
      .reset_index()
)

### Now generate charts for each ###

# 1) Donut Chart: Competence Distribution
plt.figure(figsize=(6,6))
counts = comp_counts.set_index("Competence")["Count"]
plt.pie(counts, labels=counts.index, autopct="%1.1f%%", startangle=90,
        wedgeprops=dict(width=0.4))
plt.title("Competence Distribution")
plt.tight_layout()
plt.savefig(FIG_DIR/"competence_dist_donut.png")
plt.close()

# 2) Bar: Average Score by Category
plt.figure()
avg_by_cat.set_index("Category")["Avg Score"].plot.bar()
plt.title("Average Score by Unit Category")
plt.ylabel("Avg Score")
plt.tight_layout()
plt.savefig(FIG_DIR/"avg_score_by_category.png")
plt.close()

# ─── Horizontal Bar: Pass Rate by Academic Course ─────────────────────────────

plt.figure(figsize=(10, 8))
# sort ascending so the highest‐rate courses are at the top
cp_sorted = course_pass.sort_values("Pass Rate (%)", ascending=True)
cp_sorted.set_index("Course Name")["Pass Rate (%)"].plot.barh()
plt.title("Pass Rate by Academic Course")
plt.xlabel("Pass Rate (%)")
plt.ylabel("Course Name")
plt.tight_layout()
plt.savefig(FIG_DIR/"pass_rate_by_course_hbar.png")
plt.close()


# 3) Horizontal Bar: Pass Rate by Course (Top 10)
plt.figure(figsize=(8,6))
course_pass.head(10).set_index("Course Name")["Pass Rate (%)"].plot.barh()
plt.title("Top 10 Courses by Pass Rate")
plt.xlabel("Pass Rate (%)")
plt.tight_layout()
plt.savefig(FIG_DIR/"top10_courses.png")
plt.close()

# 4) Bar: Pass Rate by Level
plt.figure()
level_pass.set_index("Level")["Pass Rate (%)"].plot.bar()
plt.title("Pass Rate by Level")
plt.ylabel("Pass Rate (%)")
plt.tight_layout()
plt.savefig(FIG_DIR/"pass_by_level.png")
plt.close()

# 5) Bar: Pass Rate by Category
plt.figure()
category_pass.set_index("Category")["Pass Rate (%)"].plot.bar()
plt.title("Pass Rate by Category")
plt.ylabel("Pass Rate (%)")
plt.tight_layout()
plt.savefig(FIG_DIR/"pass_by_category.png")
plt.close()

# 6) Grouped Bar: Pass Rate by Course & Level
# Pivot to have Levels as columns
pivot = course_level.pivot(index="Course Name", columns="Level", values="Pass_Rate").fillna(0)
pivot.plot.bar(figsize=(10,6))
plt.title("Pass Rate by Course and Level")
plt.ylabel("Pass Rate (%)")
plt.tight_layout()
plt.savefig(FIG_DIR/"pass_by_course_and_level.png")
plt.close()

# 7) Horizontal Bar: 10 Weakest Units
plt.figure(figsize=(8,6))
unit_pass.head(10).set_index("Unit Code")["Pass Rate (%)"].plot.barh()
plt.title("10 Weakest Units by Pass Rate")
plt.xlabel("Pass Rate (%)")
plt.tight_layout()
plt.savefig(FIG_DIR/"weakest_units.png")
plt.close()

# 8) Stacked Bar: Pass vs Fail Count by Course
plt.figure(figsize=(8,6))
course_stats.set_index("Course Name")[["Pass_Count","Fail_Count"]].plot.bar(stacked=True)
plt.title("Pass vs Fail Counts by Course")
plt.ylabel("Number of Unit Attempts")
plt.tight_layout()
plt.savefig(FIG_DIR/"pass_fail_counts_by_course.png")
plt.close()

print("All charts generated in:", FIG_DIR)


All charts generated in: C:\Users\DELL\Desktop\Karis\figs_all


<Figure size 800x600 with 0 Axes>

In [34]:
# ─── Cell 7: V. Performance by Course & Level & VI. Unit–Specific Analysis ─────
# V. by Course × Level
course_level = (
    df.groupby(["Course Name","Level"])
      .agg(
        Students   = ("Student Name","nunique"),
        Pass_Rate  = ("Pass","mean"),
        Fail_Rate  = ("Fail","mean")
      )
      .assign(
        Pass_Rate=lambda d: (d["Pass_Rate"]*100).round(1),
        Fail_Rate=lambda d: (d["Fail_Rate"]*100).round(1)
      )
      .reset_index()
)

# VI. Unit-Specific Performance
unit_stats = (
    df.groupby(["Unit Code","Unit Name"])
      .agg(
        Students   = ("Student Name","nunique"),
        Pass_Count = ("Pass","sum"),
        Fail_Count = ("Fail","sum")
      )
      .assign(
        Pass_Rate=lambda d: (d["Pass_Count"]/d["Students"]*100).round(1),
        Fail_Rate=lambda d: (d["Fail_Count"]/d["Students"]*100).round(1)
      )
      .reset_index()
)



# Charts

# 1) Competence distribution
plt.figure()
comp_counts.set_index("Competence")["Count"].plot.bar()
plt.title("Competence Distribution")
plt.tight_layout()
plt.savefig(FIG_DIR/"competence_dist.png")
plt.close()

# 2) Avg score by category
plt.figure()
avg_by_cat.set_index("Category")["Avg Score"].plot.bar()
plt.title("Avg Score by Category")
plt.tight_layout()
plt.savefig(FIG_DIR/"avg_score_by_category.png")
plt.close()

# 3) Pass rate by course (top 10)
plt.figure()
course_pass.head(10).set_index("Course Name")["Pass Rate (%)"].plot.barh()
plt.title("Top 10 Courses by Pass Rate")
plt.tight_layout()
plt.savefig(FIG_DIR/"top10_courses.png")
plt.close()

# 4) Pass rate by level
plt.figure()
level_pass.set_index("Level")["Pass Rate (%)"].plot.bar()
plt.title("Pass Rate by Level")
plt.tight_layout()
plt.savefig(FIG_DIR/"pass_by_level.png")
plt.close()


# 1) Donut chart: Competence Level Distribution
plt.figure(figsize=(6,6))
counts = comp_counts.set_index("Competence")["Count"]
# pie chart
patches, texts, autotexts = plt.pie(
    counts,
    labels=counts.index,
    autopct="%1.1f%%",
    startangle=90,
    wedgeprops=dict(width=0.4)  # makes it a donut
)
plt.title("Competence Level Distribution")
plt.tight_layout()
# save
plt.savefig(FIG_DIR/"competence_dist_donut.png")
plt.close()


# 5) Pass rate by course & level (one chart per level)
for lvl in course_level["Level"].unique():
    sub = course_level[course_level["Level"]==lvl]
    plt.figure()
    sub.set_index("Course Name")["Pass_Rate"].plot.barh()
    plt.title(f"Pass Rate by Course – Level {lvl}")
    plt.tight_layout()
    plt.savefig(FIG_DIR/f"pass_by_course_level_{lvl}.png")
    plt.close()

print("Summary tables and charts generated in", FIG_DIR)

print("V. Course × Level Stats (sample):")
print(course_level.head(), "\n")
print("VI. Unit Stats (sample):")
print(unit_stats.head())


Summary tables and charts generated in C:\Users\DELL\Desktop\Karis\figs_all
V. Course × Level Stats (sample):
                  Course Name Level  Students  Pass_Rate  Fail_Rate
0      Agricultural Extension     6        14       88.4       11.6
1  Building Artisan (Masonry)     4         1      100.0        0.0
2         Building Technology     6         2       62.5       37.5
3      Electrical Engineering     6         3       61.1       38.9
4     Electrical Installation     5         4       66.7       33.3 

VI. Unit Stats (sample):
            Unit Code                                          Unit Name  \
0  AGR/OS/EXT/BC/01/6                   Demonstrate Communication Skills   
1  AGR/OS/EXT/BC/02/6                        Demonstrate Numeracy Skills   
2  AGR/OS/EXT/BC/03/6                       Demonstrate Digital Literacy   
3  AGR/OS/EXT/CC/01/6   Apply Agricultural Extension and Rural Sociology   
4  AGR/OS/EXT/CR/01/6  Produce Conventional and Non-conventional Live...   

In [35]:
# Cell 8: Detailed Missing Marks Table
missing_marks = df[df["Competence"].str.upper()=="MM"].copy()
missing_marks = missing_marks[[
  "Registration Number",
  "Student Name",
  "Unit Code",
  "Unit Name"
]]
missing_marks.columns = [
  "Reg No", "Candidate Name", "Unit Code", "Unit Name"
]
missing_marks


,Reg No,Candidate Name,Unit Code,Unit Name


In [36]:
# ─── Cell 9: VIII. School-Wide Analysis per Level & Extra Counts ─────────────────
# School-wide by Level
level_stats = (
    df.groupby("Level")
      .agg(
        Total_Students = ("Student Name","nunique"),
        Total_Units    = ("Unit Code","count"),
        Pass_Rate      = ("Pass","mean"),
        Fail_Rate      = ("Fail","mean")
      )
      .assign(
        Pass_Rate=lambda d: (d["Pass_Rate"]*100).round(1),
        Fail_Rate=lambda d: (d["Fail_Rate"]*100).round(1)
      )
      .reset_index()
)

# a) # candidates per level
cand_per_level = df.groupby("Level")["Student Name"].nunique().reset_index(name="Count")

# b) per department (course)
cand_per_dept  = df.groupby("Course Name")["Student Name"].nunique().reset_index(name="Count")

# c) per level × department
cand_lvl_dept = df.groupby(["Level","Course Name"])["Student Name"]\
                  .nunique()\
                  .reset_index(name="Count")

print("VIII. Level Stats:")
print(level_stats, "\n")
print("Candidates per Level:")
print(cand_per_level, "\n")
print("Candidates per Department:")
print(cand_per_dept.head(), "\n")
print("Candidates per Level per Department:")
print(cand_lvl_dept.head())


VIII. Level Stats:
  Level  Total_Students  Total_Units  Pass_Rate  Fail_Rate
0     4               6           31       77.4       22.6
1     5              21          112       80.4       19.6
2     6              31          196       78.1       21.9 

Candidates per Level:
  Level  Count
0     4      6
1     5     21
2     6     31 

Candidates per Department:
                  Course Name  Count
0      Agricultural Extension     14
1  Building Artisan (Masonry)      1
2         Building Technology      2
3      Electrical Engineering      3
4     Electrical Installation      4 

Candidates per Level per Department:
  Level                                   Course Name  Count
0     4                    Building Artisan (Masonry)      1
1     4  Food and Beverage Production (Culinary Arts)      2
2     4                                      Plumbing      3
3     5                       Electrical Installation      4
4     5  Food and Beverage Production (Culinary Arts)      5


In [37]:
# ─── Cell 6: Compute Summaries & Save All Charts ───────────────────────────────

import matplotlib.pyplot as plt
from pathlib import Path

#–– Paths ––
BASE_DIR = Path(r"C:\Users\DELL\Desktop\Karis")
FIG_DIR  = BASE_DIR / "figs_all"
FIG_DIR.mkdir(exist_ok=True)

#–– I. Overall Metrics (Cell 5 already defined these) ––

#–– II. Competence Distribution (comp_counts) ––

#–– III–VI: course_pass, level_pass, unit_pass, category_pass –– (as before) ––

#–– VII. Missing Marks ––

#–– VIII. School‐Wide by Level ––
level_stats = (
    df.groupby("Level")
      .agg(
        Total_Students=("Student Name","nunique"),
        Total_Units   =("Unit Code","count"),
        Pass_Rate     =("Pass","mean"),
        Fail_Rate     =("Fail","mean")
      )
      .assign(
        Pass_Rate=lambda d: (d["Pass_Rate"]*100).round(1),
        Fail_Rate=lambda d: (d["Fail_Rate"]*100).round(1)
      )
      .reset_index()
)

#–– IX. Visualizations ––
# 1) Global competence dist
# Example snippet in Cell 6 that saves the bar chart
plt.figure()
comp_counts.set_index("Competence")["Count"].plot.bar()
plt.title("Competence Distribution")
plt.tight_layout()
plt.savefig(FIG_DIR/"competence_dist.png")
plt.close()


# 2) Pass rate by course (top10)
plt.figure()
course_pass.head(10).set_index("Course Name")["Pass Rate (%)"].plot.barh()
plt.title("Top 10 Courses by Pass Rate")
plt.tight_layout()
plt.savefig(FIG_DIR/"top10_courses.png")
plt.close()

# 3) Pass rate by level
plt.figure()
level_stats.set_index("Level")["Pass_Rate"].plot.bar()
plt.title("Pass Rate by Level")
plt.tight_layout()
plt.savefig(FIG_DIR/"pass_by_level.png")
plt.close()

# 4) Competence pie per level
for lvl, grp in df.groupby("Level"):
    dist = grp["Competence"].value_counts().reindex(
        ["Not Yet Competent","Competent","Proficient","Mastery"], fill_value=0
    )
    plt.figure()
    dist.plot.pie(autopct="%1.1f%%")
    plt.title(f"Competence Distribution – Level {lvl}")
    plt.ylabel("")
    plt.tight_layout()
    plt.savefig(FIG_DIR/f"pie_comp_level_{lvl}.png")
    plt.close()

# 5) Pass/Fail pie per department per level
for lvl, grp in df.groupby("Level"):
    for course, sub in grp.groupby("Course Name"):
        counts = sub["Pass"].value_counts().reindex([True,False], fill_value=0)
        plt.figure()
        plt.pie(counts, labels=["Pass","Fail"], autopct="%1.1f%%")
        plt.title(f"{course} Pass/Fail – Level {lvl}")
        plt.tight_layout()
        fname = course.replace(" ","_")[:30]
        plt.savefig(FIG_DIR/f"pie_{fname}_lvl_{lvl}.png")
        plt.close()
# ─── Append in Cell 6: Charts for Performance by Course & Level ──────────────

# course_level: DataFrame with columns ["Course Name","Level","Students","Pass_Rate","Fail_Rate"]
for lvl in course_level["Level"].unique():
    sub = course_level[course_level["Level"] == lvl]
    plt.figure()
    sub.set_index("Course Name")["Pass_Rate"].sort_values().plot.barh()
    plt.title(f"Pass Rate by Course – Level {lvl}")
    plt.xlabel("Pass Rate (%)")
    plt.tight_layout()
    plt.savefig(FIG_DIR/f"pass_by_course_level_{lvl}.png")
    plt.close()


print("All summary tables ready and charts saved to", FIG_DIR)


All summary tables ready and charts saved to C:\Users\DELL\Desktop\Karis\figs_all


In [38]:
# ─── Cell 11: X. Top Performers & Embed All Visualizations ─────────────────────

from pathlib import Path
from docx import Document
from docx.shared import Inches

# define CSV_PATH if not already in this namespace
CSV_PATH = Path(r"C:\Users\DELL\Desktop\Karis\all_candidates.csv")
BASE_DIR = CSV_PATH.parent        # C:\Users\DELL\Desktop\Karis
FIGS     = BASE_DIR / "figs2"
DOCX_PATH = BASE_DIR / "full_analysis_report.docx"

# Prepare document
doc = Document()
doc.add_heading("Comprehensive Exam Analysis", level=1)

# (… your previous code to add sections I–VIII …)



# Section IX: Visualizations
doc.add_heading("IX. Visualizations", level=2)
for img_path in sorted(FIGS.glob("*.png")):
    caption = img_path.stem.replace("_", " ").title()
    doc.add_paragraph(caption, style="Caption")
    doc.add_picture(str(img_path), width=Inches(5))

# Section X: Top Performers
dept_rates = course_stats.set_index("Course Name")["Pass_Rate"]
top_dept   = dept_rates.idxmax(), float(dept_rates.max())

unit_rates = unit_stats.set_index("Unit Code")["Pass_Rate"]
top_unit   = unit_rates.idxmax(), float(unit_rates.max())

doc.add_heading("X. Top Performers", level=2)
p = doc.add_paragraph()
p.add_run("Best Department: ").bold = True
p.add_run(f"{top_dept[0]} ({top_dept[1]:.1f}%)\n")
p.add_run("Best Unit: ").bold = True
p.add_run(f"{top_unit[0]} ({top_unit[1]:.1f}%)")

# Save
doc.save(DOCX_PATH)
print("Report saved to", DOCX_PATH)


Report saved to C:\Users\DELL\Desktop\Karis\full_analysis_report.docx


In [39]:
# ─── Cell 7: Build DOCX with Interleaved Analyses & Charts ────────────────────

from pathlib import Path
from docx import Document
from docx.shared import Inches

# Paths
BASE_DIR  = Path(r"C:\Users\DELL\Desktop\Karis")
FIG_DIR   = BASE_DIR / "figs_all"
DOCX_PATH = BASE_DIR / "full_analysis_report.docx"

doc = Document()
doc.add_heading("Comprehensive CDACC Exam Analysis", level=1)

# I. Overall Examination Statistics
doc.add_heading("I. Overall Examination Statistics", level=2)
p = doc.add_paragraph()
p.add_run("Total Students: ").bold=True; p.add_run(str(total_students)+"\n")
p.add_run("Total Units Assessed: ").bold=True; p.add_run(str(total_units)+"\n")
p.add_run("Overall Pass Rate: ").bold=True; p.add_run(f"{overall_pass_rate:.1%}\n")
p.add_run("Overall Fail Rate: ").bold=True; p.add_run(f"{overall_fail_rate:.1%}\n")

# … existing Section I code …

# Chart: Overall pass/fail distribution
chart = FIG_DIR / "overall_pass_fail_pie.png"
if chart.exists():
    doc.add_paragraph()  # some spacing
    doc.add_picture(str(chart), width=Inches(5))

# II. Competence Level Distribution & Counts
doc.add_heading("II. Competence Level Distribution & Counts", level=2)
tbl = doc.add_table(rows=1, cols=2)
hdr = tbl.rows[0].cells; hdr[0].text, hdr[1].text = "Competence", "Count"
for _, r in comp_counts.iterrows():
    c = tbl.add_row().cells; c[0].text, c[1].text = r["Competence"], str(r["Count"])
# total row
c = tbl.add_row().cells; c[0].text, c[1].text = "Total Units Assessed", str(total_units)
# Chart: donut for distribution
chart = FIG_DIR/"competence_dist_donut.png"
if chart.exists():
    doc.add_paragraph(); doc.add_picture(str(chart), width=Inches(5))

# III. Performance by Unit Category (Avg Score)
doc.add_heading("III. Performance by Unit Category (Avg Score)", level=2)
tbl = doc.add_table(rows=1, cols=2)
hdr = tbl.rows[0].cells; hdr[0].text,hdr[1].text = "Category","Avg Score"
for _, r in avg_by_cat.iterrows():
    c = tbl.add_row().cells; c[0].text,c[1].text = r["Category"], str(r["Avg Score"])
# Chart: bar of avg score
chart = FIG_DIR/"avg_score_by_category.png"
if chart.exists():
    doc.add_paragraph(); doc.add_picture(str(chart), width=Inches(5))

# IV. Performance by Academic Course
doc.add_heading("IV. Performance by Academic Course", level=2)
tbl = doc.add_table(rows=1, cols=3)
hdr = tbl.rows[0].cells
hdr[0].text,hdr[1].text,hdr[2].text = "Course","Pass Rate (%)","Fail Rate (%)"
for _, r in course_stats.iterrows():
    c = tbl.add_row().cells
    c[0].text,c[1].text,c[2].text = r["Course Name"], str(r["Pass_Rate"]), str(r["Fail_Rate"])
# Chart: barh of pass rate
chart = FIG_DIR/"pass_rate_by_course.png"
if chart.exists():
    doc.add_paragraph(); doc.add_picture(str(chart), width=Inches(5))

# ─── Embed the Course Performance chart under Section IV ──────────────────────

doc.add_heading("IV. Performance by Academic Course", level=2)

# (your table code here…)

# now embed the horizontal‐bar chart
chart = FIG_DIR / "pass_rate_by_course_hbar.png"
if chart.exists():
    doc.add_paragraph()
    doc.add_picture(str(chart), width=Inches(6))


# V. Performance by Academic Course & Level
doc.add_heading("V. Performance by Academic Course & Level", level=2)
tbl = doc.add_table(rows=1, cols=4)
hdr = tbl.rows[0].cells
hdr[0].text,hdr[1].text,hdr[2].text,hdr[3].text = "Course","Level","Students","Pass Rate (%)"
for _, r in course_level.iterrows():
    c = tbl.add_row().cells
    c[0].text,c[1].text,c[2].text,c[3].text = (
        r["Course Name"], r["Level"], str(r["Students"]), str(r["Pass_Rate"])
    )
# Chart: grouped bar chart
chart = FIG_DIR/"pass_by_course_and_level.png"
if chart.exists():
    doc.add_paragraph(); doc.add_picture(str(chart), width=Inches(5))

# VI. Unit-Specific Performance
doc.add_heading("VI. Unit-Specific Performance", level=2)
tbl = doc.add_table(rows=1, cols=3)
hdr = tbl.rows[0].cells
hdr[0].text,hdr[1].text,hdr[2].text = "Unit Code","Unit Name","Pass Rate (%)"
for _, r in unit_pass.iterrows():
    c = tbl.add_row().cells
    c[0].text,c[1].text,c[2].text = r["Unit Code"], r["Unit Name"], str(r["Pass Rate (%)"])
# Chart: weakest units barh
chart = FIG_DIR/"weakest_units.png"
if chart.exists():
    doc.add_paragraph(); doc.add_picture(str(chart), width=Inches(5))

# … earlier in Cell 7 …

# VII. Missing Marks Identification (detailed)
doc.add_heading("VII. Missing Marks Identification", level=2)

# 1) Build the detailed missing‐marks table
missing_marks = (
    df[df["Competence"].str.upper() == "MM"]  # filter MM rows
      .loc[:, ["Registration Number","Student Name","Unit Code","Unit Name"]]
      .copy()
)
# Rename columns for the report
missing_marks.columns = ["Reg No","Candidate Name","Unit Code","Unit Name"]

# 2) Insert into the DOCX
if not missing_marks.empty:
    tbl = doc.add_table(rows=1, cols=4)
    hdr = tbl.rows[0].cells
    hdr[0].text, hdr[1].text, hdr[2].text, hdr[3].text = (
        "Reg No","Candidate Name","Unit Code","Missed Unit Name"
    )
    for _, r in missing_marks.iterrows():
        cells = tbl.add_row().cells
        cells[0].text = r["Reg No"]
        cells[1].text = r["Candidate Name"]
        cells[2].text = r["Unit Code"]
        cells[3].text = r["Unit Name"]
else:
    doc.add_paragraph("No missing marks detected.")

# … continue with Section VIII …


# VIII. School-Wide Analysis per Level
doc.add_heading("VIII. Analysis per Level", level=2)
tbl = doc.add_table(rows=1, cols=5)
hdr = tbl.rows[0].cells
hdr[0].text,hdr[1].text,hdr[2].text,hdr[3].text,hdr[4].text = (
    "Level","Students","Units","Pass Rate (%)","Fail Rate (%)"
)
for _, r in level_stats.iterrows():
    c = tbl.add_row().cells
    c[0].text,c[1].text,c[2].text,c[3].text,c[4].text = (
        r["Level"], str(r["Total_Students"]), str(r["Total_Units"]),
        str(r["Pass_Rate"]), str(r["Fail_Rate"])
    )
# Chart: pie competence per level first one
chart = FIG_DIR/"pass_by_level.png"
if chart.exists():
    doc.add_paragraph(); doc.add_picture(str(chart), width=Inches(5))

# IX. Visualizations by Level and Department
doc.add_heading("IX. Analysis by Level", level=2)
# competence pies per level
for lvl in df["Level"].unique():
    chart = FIG_DIR/f"pie_comp_level_{lvl}.png"
    if chart.exists():
        doc.add_heading(f"Competence – Level {lvl}", level=3)
        doc.add_picture(str(chart), width=Inches(4))
# pass/fail pies per department per level
doc.add_heading("IX. Analysis by Level and Department", level=2)
for img in FIG_DIR.glob("pie_*_lvl_*.png"):
    doc.add_paragraph(); doc.add_picture(str(img), width=Inches(4))

# X. Top Performers
# doc.add_heading("X. Top Performers", level=2)
#best_dept = course_pass.set_index("Course Name")["Pass Rate (%)"].idxmax()
#best_dept_rate = float(course_pass.set_index("Course Name")["Pass Rate (%)"].max())
#best_unit = unit_pass.set_index("Unit Code")["Pass Rate (%)"].idxmax()
#best_unit_rate = float(unit_pass.set_index("Unit Code")["Pass Rate (%)"].max())
#p = doc.add_paragraph()
#p.add_run("Best Department: ").bold=True; p.add_run(f"{best_dept} ({best_dept_rate:.1f}%)\n")
#p.add_run("Best Unit: ").bold=True; p.add_run(f"{best_unit} ({best_unit_rate:.1f}%)\n")

# Save
doc.save(DOCX_PATH)
print("Report with interleaved analyses & charts saved to:", DOCX_PATH)


Report with interleaved analyses & charts saved to: C:\Users\DELL\Desktop\Karis\full_analysis_report.docx
